In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
file_path = r'C:\Users\bisht\Downloads\census+income\adult.data'

df = pd.read_csv(file_path, header=None)
col=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','Income']
len(col)

15

In [26]:
df.columns=col
df = df.applymap(lambda x: np.nan if str(x).strip() == '?' else x)
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


# Data Preprocessing  

In [27]:
df=df.drop(columns=['fnlwgt'])

In [28]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder

In [30]:
num_random_values = df['occupation'].isnull().sum()
possible_categories = df['occupation'].dropna().unique()
random_values = np.random.choice(possible_categories,num_random_values)
df.loc[df['occupation'].isnull(),'occupation'] = random_values

In [31]:
num=[0,1,4,5,6,7,12]
col = df.columns[num]
for i in col:
    dictt=df[i].value_counts().to_dict()
    df[i]=df[i].map(dictt)
df

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Income
0,816,1298.0,Bachelors,13,10683,3897,8305,27816,Male,2174,0,40,29170.0,<=50K
1,602,2541.0,Bachelors,13,14976,4221,13193,27816,Male,0,0,13,29170.0,<=50K
2,827,22696.0,HS-grad,9,4443,1490,8305,27816,Male,0,0,40,29170.0,<=50K
3,464,22696.0,11th,7,14976,1490,13193,3124,Male,0,0,40,29170.0,<=50K
4,867,22696.0,Bachelors,13,14976,4286,1568,3124,Female,0,0,40,95.0,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,835,22696.0,Assoc-acdm,12,14976,1049,1568,27816,Female,0,0,38,29170.0,<=50K
32557,794,22696.0,HS-grad,9,14976,2128,13193,27816,Male,0,0,40,29170.0,>50K
32558,366,22696.0,HS-grad,9,993,3897,3446,27816,Female,0,0,40,29170.0,<=50K
32559,765,22696.0,HS-grad,9,10683,3897,5068,27816,Male,0,0,20,29170.0,<=50K


In [32]:
imputer = SimpleImputer(strategy='most_frequent')

In [33]:
trf1 = ColumnTransformer(
    transformers=[
        ('Imputation', imputer, ['native-country', 'workclass'])
    ]
)

In [34]:
subset=trf1.fit_transform(df)
subset=pd.DataFrame(subset,columns=['native-country', 'workclass'])

In [35]:
df[['native-country','workclass']]=subset

In [36]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [37]:
le=LabelEncoder()
y=le.fit_transform(y)
y=pd.DataFrame(y,columns=['Income'])

In [38]:
Oe=OrdinalEncoder(categories=[['Preschool', '1st-4th', '5th-6th', '7th-8th', '9th', '10th', '11th', '12th', 'HS-grad', 'Some-college', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Masters', 'Doctorate', 'Prof-school']]
)

In [39]:
X['education']=Oe.fit_transform(X[['education']])

In [40]:
ohe=OneHotEncoder(drop='first',handle_unknown='ignore',sparse=False)
ohe=ohe.fit_transform(X[['sex']])
X['sex']=pd.DataFrame(ohe)

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Balancing Dataset

In [43]:
y.value_counts()

Income
0         24720
1          7841
dtype: int64

In [44]:
#### Using smot
from imblearn.over_sampling import SMOTE

In [45]:
smote=SMOTE(sampling_strategy='minority')

In [46]:
X_sm,y_sm=smote.fit_resample(X,y)

In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_sm,y_sm,random_state=52,test_size=0.2,stratify=y_sm)

# Feature Selection

In [67]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [68]:
sfs=SFS(RandomForestClassifier(),k_features='best',forward=False,floating=False,scoring='accuracy',cv=5)

In [69]:
sfs.fit(X_train,y_train)

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

...

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y wa

SequentialFeatureSelector(estimator=RandomForestClassifier(), forward=False,
                          k_features=(1, 13), scoring='accuracy')

In [70]:
sfs.k_feature_idx_

(0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12)

In [71]:
metrics_df=pd.DataFrame.from_dict(sfs.get_metric_dict()).T

In [73]:
metrics_df['observation']=X.shape[0]
metrics_df['num_features']=metrics_df['feature_idx'].apply(lambda x:len(x))

In [74]:
metrics_df

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err,observation,num_features
13,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","[0.8893945139678928, 0.8901529515863987, 0.883...",0.889083,"(age, workclass, education, education-num, mar...",0.004006,0.003117,0.001559,32561,13
12,"(0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12)","[0.8910377954746556, 0.8911642017444065, 0.882...",0.888981,"(age, workclass, education, marital-status, oc...",0.005088,0.003958,0.001979,32561,12
11,"(0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12)","[0.8897737327771458, 0.8919226393629124, 0.883...",0.889487,"(age, workclass, education, occupation, relati...",0.004239,0.003298,0.001649,32561,11
10,"(0, 1, 2, 5, 6, 7, 8, 9, 10, 11)","[0.8869927948426242, 0.8915434205536594, 0.880...",0.887363,"(age, workclass, education, occupation, relati...",0.005459,0.004247,0.002123,32561,10
9,"(0, 1, 2, 5, 6, 7, 9, 10, 11)","[0.8869927948426242, 0.8844646694476046, 0.880...",0.884329,"(age, workclass, education, occupation, relati...",0.002888,0.002247,0.001123,32561,9
8,"(0, 1, 2, 5, 6, 9, 10, 11)","[0.8807988876248262, 0.8832006067500948, 0.876...",0.881295,"(age, workclass, education, occupation, relati...",0.003785,0.002945,0.001472,32561,8
7,"(0, 1, 2, 5, 6, 9, 10)","[0.8762482619137909, 0.875616230565036, 0.8726...",0.876466,"(age, workclass, education, occupation, relati...",0.003218,0.002503,0.001252,32561,7
6,"(0, 1, 2, 6, 9, 10)","[0.875489824295285, 0.8725824800910125, 0.8734...",0.873812,"(age, workclass, education, relationship, capi...",0.003943,0.003067,0.001534,32561,6
5,"(0, 2, 6, 9, 10)","[0.8809252938945772, 0.8735937302490203, 0.880...",0.879172,"(age, education, relationship, capital-gain, c...",0.004576,0.00356,0.00178,32561,5
4,"(0, 2, 6, 9)","[0.8733409177095184, 0.8662621666034636, 0.875...",0.872624,"(age, education, relationship, capital-gain)",0.004993,0.003885,0.001942,32561,4


In [77]:
sfs1=SFS(KNeighborsClassifier(),k_features='best',forward=False,floating=False,scoring='accuracy',cv=5)

In [78]:
sfs1.fit(X_train,y_train)
sfs1.k_feature_idx_

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

...

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sam

(2, 3, 5, 6, 8, 9, 10)

In [81]:
metrics_df1=pd.DataFrame.from_dict(sfs1.get_metric_dict()).T
metrics_df1

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
13,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)","[0.8379471621792441, 0.838073568448995, 0.8259...",0.83442,"(age, workclass, education, education-num, mar...",0.005861,0.00456,0.00228
12,"(0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12)","[0.8366830994817342, 0.8406016938440146, 0.827...",0.835356,"(age, workclass, education, education-num, occ...",0.005645,0.004392,0.002196
11,"(0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12)","[0.8337757552774617, 0.8416129440020225, 0.828...",0.835937,"(age, workclass, education, education-num, occ...",0.005913,0.004601,0.0023
10,"(1, 2, 3, 5, 6, 8, 9, 10, 11, 12)","[0.8328909113892049, 0.8349134117052206, 0.835...",0.835887,"(workclass, education, education-num, occupati...",0.002721,0.002117,0.001059
9,"(2, 3, 5, 6, 8, 9, 10, 11, 12)","[0.8375679433699912, 0.8416129440020225, 0.840...",0.841475,"(education, education-num, occupation, relatio...",0.003594,0.002796,0.001398
8,"(2, 3, 5, 6, 8, 9, 10, 12)","[0.8412337251927695, 0.8503349766148401, 0.842...",0.847062,"(education, education-num, occupation, relatio...",0.00596,0.004637,0.002318
7,"(2, 3, 5, 6, 8, 9, 10)","[0.8534951333586146, 0.8558968524838833, 0.845...",0.852296,"(education, education-num, occupation, relatio...",0.005407,0.004207,0.002103
6,"(2, 3, 5, 6, 9, 10)","[0.8531159145493616, 0.8537479458981165, 0.842...",0.850602,"(education, education-num, occupation, relatio...",0.006012,0.004677,0.002339
5,"(2, 5, 6, 9, 10)","[0.850967007963595, 0.8523574769308557, 0.8403...",0.848857,"(education, occupation, relationship, capital-...",0.00657,0.005112,0.002556
4,"(2, 5, 6, 9)","[0.850461382884591, 0.8459107571735558, 0.8319...",0.843446,"(education, occupation, relationship, capital-...",0.010112,0.007868,0.003934


In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [60]:
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)

C:\Users\bisht\anaconda3\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [61]:
y_pred=knn.predict(X_test)

In [62]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       0.82      0.87      0.84      4620
           1       0.88      0.83      0.85      5268

    accuracy                           0.85      9888
   macro avg       0.85      0.85      0.85      9888
weighted avg       0.85      0.85      0.85      9888



In [63]:
accuracy_score(y_pred,y_test)

0.8491100323624595

In [64]:
from sklearn.ensemble import RandomForestClassifier
rfg=RandomForestClassifier()
rfg.fit(X_train,y_train)

C:\Users\bisht\AppData\Local\Temp\ipykernel_10772\4252362567.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfg.fit(X_train,y_train)


RandomForestClassifier()

In [65]:
y_pred=rfg.predict(X_test)

In [66]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_pred,y_test))


              precision    recall  f1-score   support

           0       0.90      0.89      0.90      4992
           1       0.89      0.90      0.90      4896

    accuracy                           0.90      9888
   macro avg       0.90      0.90      0.90      9888
weighted avg       0.90      0.90      0.90      9888

